In [ ]:
%run Utils.ipynb

In [ ]:
class Verifier_first_stage():
    
    def __init__(self):
        """
        """
    
    def verify(self, proof_stage_one: dict, domian_size: int):
            index = self.get_index(proof_stage_one=proof_stage_one, domian_size=domian_size)
            self.verifiy_path_and_values(proof_stage_one=proof_stage_one, index=index, domian_size=domian_size)
            coeffs = self.get_coefficients(proof_stage_one=proof_stage_one)
            self.validate_cp_value_in_index(proof_stage_one=proof_stage_one, coeffs=coeffs)

    def verifiy_path_and_values(self, proof_stage_one: dict, index: int, domian_size: int):
        names = ['p1', 'p2', 'p3', 'cp']
        for name in names:
            internal_proof_dict = proof_stage_one[name]
            root = internal_proof_dict['root']
            value_at_index = internal_proof_dict['value']
            path = internal_proof_dict['path']
            self.verify_path_by_index(
                root=root, expected_value=value_at_index, index=index, domain_size=domian_size, path=path
            )
            
    def get_index(self, proof_stage_one: dict, domian_size: int):
        #print(f"{proof_stage_one['cp']=}")
        cp_tree_root = proof_stage_one['cp']['root']
        return (fiat_shamir_random(cp_tree_root)%domian_size)
    
    def get_coefficients(self, proof_stage_one:dict) -> dict:
        names = ['p1', 'p2', 'p3']
        return {name:fiat_shamir_random(proof_stage_one[name]['root']) for name in names}
    
    def validate_cp_value_in_index(self, proof_stage_one: dict, coeffs: dict):
        names = ['p1', 'p2', 'p3']
        expected_value = 0
        for name in names:
            expected_value_int = ((proof_stage_one[name]['value'])[1]*coeffs[name])
            expected_value += expected_value_int%PRIME            
        assert proof_stage_one['cp']['value'][1] == expected_value
    
    def verify_path_by_index(self, root: str, expected_value: tuple, index: int, domain_size: int, path: dict):

        index_size = int(log(domain_size, 2))
        index_as_str = format(index, f'#0{index_size+2}b')
        # shift from '0b1110' to '1110'
        index_as_str = index_as_str[2:] 
        key=root
        while(index_as_str):
            value = path[key]

            #Verify the hash:
            assert hash_tow_elements(value[0], value[1]) == key

            direction_bit = int(index_as_str[0])
            key = value[direction_bit]
            index_as_str = index_as_str[1:]

        #Now the key is the hash of the required index. Reauired value = tree[key] = (coset, CP[coset])
        value = path[key]
        assert hash_one_elements(value) == key
        assert value == expected_value